# 1. ***Introduction*** 
This notebook is intended to provide a simple model to predict the Scaled sound pressure using 4 features.

First we import all the necessary packages

In [1]:
from pandas import read_csv
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.compat.v1 as tf
import keras
from sklearn.linear_model import LinearRegression
from keras.models import Sequential
from sklearn.preprocessing import PolynomialFeatures
#tf.disable_v2_behavior()

We start by importing the data and identifying the potential opportunities to reduce dimensionality

In [ ]:
!pip3 install -U scikit-learn==0.24

In [2]:
url = "/kaggle/input/airfoil-selfnoise-dataset/AirfoilSelfNoise.csv"
data = read_csv(url)
print(data)

In [3]:
data = data.rename(columns={"f": 'Frequency',"alpha":'Angle of attack',"c":'Chord length',"U_infinity":'Free-stream velocity',"delta":"Section side displacement thickness","SSPL":"Scaled Sound Pressure Level"})
print(data)

In [4]:
data.describe()

# 3. ***Univariate analysis***  

In [5]:
sns.distplot(a=data['Frequency'], hist=False)
plt.show()
sns.distplot(a=data['Angle of attack'], hist=False)
plt.show()
sns.distplot(a=data['Chord length'], hist=False)
plt.show()
sns.distplot(a=data['Free-stream velocity'], hist=False)
plt.show()
sns.distplot(a=data['Section side displacement thickness'], hist=False)
plt.show()
sns.distplot(a=data['Scaled Sound Pressure Level'], hist=False)
plt.show()




 


We could notice in case of the variable frequency a wide range of frequency values that are rare and could potentially indicate a noisy record. A general trend is that lower values of each feature are most frequent except for the output variable.Chord length and free stream velocity density curves are less smoother then the other variables density curves, and so presents many peaks in terms of density which means that some ranges of chords length values are distributed almost equally compared to other ranges. free stream velocity presents three peaks which corresponds to 3 ranges of velocity that are mostly frequent. We are now interested in these two variable along with angle of attacks.

# 3. ***Bivariate analysis***  

Next, I will only show some interesting relations between features.
In case of frequency and Section side displacement thickness, we could notice that high airfoil noise frequencies only fits low displacement thickness.


In [6]:
fig,ax= plt.subplots()
plt.style.use('ggplot')
ax.plot(data["Frequency"],data["Section side displacement thickness"])
ax.set_title('My Chart')
ax.set_xlabel('Frequency')
ax.set_ylabel('Section side displacement thickness')


The relation between Frequency and displacement thickness seems at first linear.

In [7]:
fig,ax= plt.subplots()
plt.style.use('ggplot')
ax.plot(data["Angle of attack"],data["Section side displacement thickness"])
ax.set_title('My Chart')
ax.set_xlabel('Frequency')
ax.set_ylabel("Section side displacement thickness")


The following code is imported from the notebook https://www.kaggle.com/code/shtrausslearning/gaussian-processes-airfoil-noise-modeling/notebook. The author presents some interesting experiments about the data. The author visualizes the evolution of scaled sound pressure regarding noise frequency for different configs of attack angle while also experimenting with different configs of chord length and stream velocity. The authors keep chord length constant in both experiments and varies stream velocity. This type of plots could be very useful to : understand the effect of one explanotory variable over the dependent variable, discorver general trend in the data, discover patterns, which variable decides the most on the depedent variable. My role would be to reinterpret the content of these plots. 

In [8]:
def plotRel(ldf):
    sns.set(style='whitegrid')
    g = sns.relplot(x='Frequency',y='Scaled Sound Pressure Level',col='velocity',row='Chord length',data=ldf,
                kind='line',legend='full',hue='Angle of attack',palette='jet_r',height=4)


In [9]:
data = data.rename(columns={"Frequency": 'Frequency',"Angle of attack":'Angle of attack',"Chord length":'Chord length',"Free-stream velocity":'velocity',"Section side displacement thickness":"Section side displacement thickness","Scaled Sound Pressure Level":"Scaled Sound Pressure Level"})
print(data)

''' Create Data Subsets for different airfoil sizes '''
sns.set(style='whitegrid')
pdlst_lchord = [] # list of pd subset data storing all unique 'l_chord'
lst_lchord_unique = data['Chord length'].unique().tolist()
lst_lchord_unique.sort() # get unique 'l_chord'&sort()
for i in lst_lchord_unique:
    ldf = data[data['Chord length']==i].copy()
    pdlst_lchord.append(ldf)    # add all unique subsets to list


In [10]:
# lets plot two cases, they are ordered so smaller one on top.
plotRel(pdlst_lchord[0])
plotRel(pdlst_lchord[2])

Firstly, we could notice some general tendencies in the data :

Starting from low noise frequencies, every experiment shows **sudden**  peak sound pressure, then a quick downfall of pressure, as we move up in noise frequency a **smooth** peak sound pressure followed by a smooth decrease in pressure for high frequencies. This could insinuate that specific ranges of noise frequency values indicates often that sound pressure will attain its highest value. high frequency values often indicate low sound pressure compared to the average.


chord length = 0.0254
Considering frequencies below 2500 Hz, we could easily infer that high angles of attacks usually produce the highest wave of sound pressure.
Considering frequencies starting from 2500 Hz, we could easily infer that very low angle of attacks usually produce the highest wave of sound pressure.

chord length = 0.1016
Considering frequencies below 2500 Hz, we could easily infer that medium angles of attacks usually produce the highest wave of sound pressure.
Considering frequencies starting from 2500 Hz, we could easily infer that low angle of attacks usually produce the highest wave of sound pressure.


Next the author studies boundary layer thickness

In [11]:
sns.set(style='whitegrid')
lst = [0,2]
def relplot1(id):
    for i in id:
        g = sns.relplot(x='Scaled Sound Pressure Level',y='Section side displacement thickness',col='velocity',row='Chord length',
                    hue='Angle of attack',sizes='Scaled Sound Pressure Level',palette='jet_r',
                    kind='scatter',legend='full',data=pdlst_lchord[i])
        g.fig.set_size_inches(16,3)
        leg = g._legend
        leg.set_bbox_to_anchor([1.06,1])  # coordinates of lower left of bounding box
        leg._loc = 1  # if required you can set the loc

In [ ]:
relplot1(lst)

the first remark we could draw from the plots is the range of values of Boundary layer thickness. In the first case of chord length = 0.0254 the range of variations is pretty small (0.022) compared to that of chord length = 0.1016 where the thickness ranges in a wider intervall (0.051)

chord length = 0.0254 
The records of very low boundary layer tickness (0.000 -- 0.005) were always observed in cases of low angles of attack (0.0 -- 6.7).

chord length = 0.1016 
The records of very low boundary layer tickness (0.00 -- 0.01) were always observed in cases of low angles of attack (0.0 -- 6.7).

We also notice two different patterns in cases where velocity values (31.7,55.5) and (39.6,71.3).

In both cases of chord length, we notice the inexistance of records where the thickness equals 12.7 or 22.2. It's like two periodic patterns that reproduce themselves alternately.

# 4. ***Feature extraction***  

Let's identify potential opportunities to reduce dimensionality. We notice a relatively strong corrolation between the feature Angle of attack,Section side displacement thickness.

In [12]:
print(data.corr())

We apply a dimension reduction technique to replace the feature Angle of attack, Section side displacement thickness by a principle component that best represents them, as it's clear,this new componenent explains almost all variance of these two features and best represents them, it would make sense to omit these two feature and introduce the principle component instead.

In [13]:
x_org = np.array(data[["Frequency","Chord length","velocity","Angle of attack","Section side displacement thickness"]])
y_org = np.array(data[["Scaled Sound Pressure Level"]])
x_train_org, x_test_org, y_train_org, y_test_org = train_test_split(x_org, y_org,train_size=0.85)
scaler = StandardScaler()
# transform original data
x_train_org = scaler.fit_transform(x_train_org)
x_test_org = scaler.fit_transform(x_test_org)

As we can see the new component omitted some of the colinearity.

In [14]:
pca = PCA(n_components=1)
pca.fit(np.array(data[["Angle of attack","Section side displacement thickness"]]))
print("variance explained : ",pca.explained_variance_ratio_)

y = pca.transform(np.array(data[["Angle of attack","Section side displacement thickness"]]))
new_component = pd.DataFrame(y)
new_component = new_component.rename(columns={0: 'New component'})
data["New component"] = new_component 
del data["Angle of attack"]
del data["Section side displacement thickness"]

In [15]:
print(data.corr())

In [16]:
x = np.array(data[["Frequency","Chord length","velocity","New component"]])
y = np.array(data[["Scaled Sound Pressure Level"]])
x_train, x_test, y_train, y_test = train_test_split(x, y,train_size=0.85)
scaler = StandardScaler()
# transform  data
x_train = scaler.fit_transform(x_train)
x_test = scaler.fit_transform(x_test)


# ****3. Modelling****

# ***3.1 ANN***
We prepare a simple neural network model for both versions of datasets the original one (5 features), and the new one where we transform two features into one representative features.

In [18]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(4,)))
model.add(tf.keras.layers.Dense(units=300,activation="sigmoid"))
model.add(tf.keras.layers.Dense(units=300,activation="sigmoid"))
model.add(tf.keras.layers.Dense(units=1))
model.summary()


Next we train the model on the training dataset using the SGD solver. the loss function evolution plot shows a quick convergence to the minimum. In fact only 20 epochs are needed to achieve this convergence, still the loss function best value equals 48.3528 at the end of the training process which means the models didn't completely fitted the data. The risk of overfitting here isn't iminent because the loss function values for both training and validation sets aren't too far from each other. We choose a model that as complex as the problem itself.

In [19]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),loss="mean_squared_error",metrics=["mean_absolute_percentage_error"])
history = model.fit(x_train, y_train,batch_size=1277,epochs=300,steps_per_epoch=1,validation_data=(x_test, y_test))
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('training loss')
plt.xlabel('epoch')
plt.show()
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('validation loss')
plt.xlabel('epoch')
plt.show()

In [33]:
model = tf.keras.models.Sequential()
model.add(tf.keras.Input(shape=(5,)))
model.add(tf.keras.layers.Dense(units=3,activation="sigmoid"))
model.add(tf.keras.layers.Dense(units=4,activation="sigmoid"))
model.add(tf.keras.layers.Dense(units=1))
model.summary()

In [34]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),loss="mean_squared_error",metrics=["mean_absolute_percentage_error"])
history = model.fit(x_train_org, y_train_org,batch_size=1277,epochs=300,steps_per_epoch=1,validation_data=(x_test_org, y_test_org))
plt.plot(history.history['loss'])
plt.title('model loss')
plt.ylabel('training loss')
plt.xlabel('epoch')
plt.show()
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('validation loss')
plt.xlabel('epoch')
plt.show()

In [35]:
ANN_predictions = model.predict(x_test_org)


ANN:
mean_absolute_percentage_error of the validation data: 3.40

mean_absolute_percentage_error of the validation 4-features data (in this case we used the data where we converted angle of attack and thickness to one representative column) (4 features) : 3.52

# ***3.2 Linear regression***

In [21]:
def MeanAbsolutePercentageError(y_true,y_predicted):
    S = 0
    for i in range(0,len(y_true[0])):
        S = S + abs((y_true[i] - y_predicted[i])/y_true[i])
    return 100*S/len(y_true[0])
    

In [37]:
LR = LinearRegression()
LR.fit(x_train_org,y_train_org)
y_prediction_LR =  LR.predict(x_test_org)
metric = MeanAbsolutePercentageError(y_test_org,y_prediction_LR)
print("Mean absolute percentage error :")
print(metric[0])
print("R square :")
R_square =  LR.score(x_test_org,y_test_org)
print(R_square)

Linear regression:

mean_absolute_percentage_error of the validation data : 1.165



# ***3.2 Polynomial Regression***

In [38]:
poly = PolynomialFeatures(2)
new_features_train = poly.fit_transform(x_train_org)
new_features_test = poly.fit_transform(x_test_org)


In [39]:
LR = LinearRegression()
LR.fit(new_features_train,y_train_org)
y_prediction_PR2 =  LR.predict(new_features_test)
metric = MeanAbsolutePercentageError(y_test_org,y_prediction_PR2)
print("Polynomial degree : 2")
print("Mean absolute percentage error :")
print(metric[0])
print("R square :")
R_square =  LR.score(new_features_test,y_test_org)
print(R_square)

Polynomial regression of degree 2:

mean_absolute_percentage_error of the validation data : 1.165

In [40]:
poly = PolynomialFeatures(3)
new_features_train = poly.fit_transform(x_train_org)
new_features_test = poly.fit_transform(x_test_org)


In [41]:
LR = LinearRegression()
LR.fit(new_features_train,y_train_org)
y_prediction_PR3 =  LR.predict(new_features_test)
metric = MeanAbsolutePercentageError(y_test_org,y_prediction_PR3)
print("Polynomial degree : 3")
print("Mean absolute percentage error :")
print(metric[0])
print("R square :")
R_square =  LR.score(new_features_test,y_test_org)
print(R_square)

Polynomial regression of degree 3:

mean_absolute_percentage_error of the validation data : 1.72

# ****4. Synthesis****

In [59]:
predictions = {'ANN':np.transpose(ANN_predictions)[0],
        'Linear Regression': np.transpose(y_prediction_LR)[0],
        'Polynomial Regression degree 2': np.transpose(y_prediction_PR2)[0],
        "Polynomial Regression degree 3":np.transpose(y_prediction_PR3)[0],
        "original validation set":np.transpose(y_test_org)[0]}

predictions = pd.DataFrame(predictions)

chart = sns.displot(data=predictions, kind='kde', fill=True, palette=sns.color_palette('bright')[:5], height=5, aspect=1.5)

## Changing title
new_title = 'Different models output distributions'
chart._legend.set_title(new_title)

# Replacing labels
new_labels = ['ANN', 'Linear Regression', 'Polynomial Regression degree 2','Polynomial Regression degree 3',"original validation set distribution"]
for t, l in zip(chart._legend.texts, new_labels):
    t.set_text(l)

The X axis presents the values of airfoil sound pressure.

We could notice that degree 3 Polynomial regression model was the closest model to imitate the sound pressure probability distribution. The other models didn't really reflect the data distribution such as ANN which was the furthest one from reality followed by LR,PR degree 2 ,PR degree 3.